In [186]:
# General Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Web Scraping Libraries
import urllib
import requests
from bs4 import BeautifulSoup

# Regex Library
import re

# Time-related Libraries
import time
import datetime

# NLP Libraries
import unicodedata
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
from wordcloud import WordCloud

# Helper functions
import MVP_Bojado, MVP_Shi

# Environment file
import env, env_Shi

# AWS
import logging
import boto3
from botocore.exceptions import ClientError

import json

import warnings
warnings.filterwarnings("ignore")

## Data Acquisition

<div class="alert alert-block alert-success"><b>All the functions in the Data Acquisitioin section have been tested out and inorporated into the MVP_acquire_ds.py and MVP_acquire_wd.py files. To save space, no extra test is carried out in this notebook.</b></div>

### URL Format of Indeed.com
1. Search chemist in TX<br>
https://www.indeed.com/jobs?q=chemist&l=TX
2. Search chemist in San Antonio, TX<br>
https://www.indeed.com/jobs?q=chemist&l=San+Antonio%2C+TX
3. Search data scientist in San Antonio, TX<br>
https://www.indeed.com/jobs?q=data+scientist&l=San+Antonio%2C+TX
4. Search data scientist intern in San Anotnio, TX<br>
https://www.indeed.com/jobs?q=data+scientist+intern&l=San+Antonio%2C+TX
5. Sort the data scientist jobs posting by date<br>
https://www.indeed.com/jobs?q=data+scientist&l=San+Antonio%2C+TX&sort=date

**Takeaways**
1. q = job title
2. l = location

### URL Format of Monster.com
https://www.monster.com/jobs/search/?q=data-scientist&where=San-Antonio__2C-TX

### Generate the URL of a Job Search at Indeed.com

In [ ]:
def first_page_url_indeed(job_title, location):
    '''
    This function returns a URL of the 1st page of a job search at Indeed.com 
    based on the job title and the location.
    '''
    # Create the base URL for a job serch at Indeed.com
    base_url = 'https://www.indeed.com/jobs?'
    # Create a dictionary to map the keys to the input parameters
    dic = {'q': job_title, 'l': location, 'sort': 'date'}
    # Convert the dictionary to a query string
    relative_url = urllib.parse.urlencode(dic)
    # Generate the full URL of the first page
    url = base_url + relative_url
    return url

### Make the HTTP Request

In [ ]:
def first_page_soup_indeed(job_title, location):
    '''
    This function returns a BeautifulSoup object to hold the content 
    of the first page of a request for job searching at Indeed.com
    '''
    # Generate the URL of the job search based on title and location
    url = first_page_url_indeed(job_title, location)
    # Make the HTTP request
    response = requests.get(url)
    # Print the status code of the request
    print("Status code of the request: ", response.status_code)
    # Sanity check to make sure the document type is HTML
    print("Document type: ", response.text[:15])
    # Take a break
    time.sleep(5)
    # Make a soup to hold the response content
    soup = BeautifulSoup(response.content, "html.parser")
    # Print out the title of the content
    print("Title of the response: ", soup.title.string)
    return soup

In [ ]:
# first_page_soup = first_page_soup_indeed("data scientist", 'al')
# type(first_page_soup)

# # Find out the tag that contains the number of the jobs by seaching

# num_jobs = first_page_soup.find('div', id='searchCountPages')
# print("Data Type: ", type(num_jobs))
# print("Name of the Tag: ", num_jobs.name)
# print("Attributes of the Tag: ", num_jobs.attrs)
# print("Text within the Tag: ")
# num_jobs.text

# # Find the number of the jobs in the text
# match = re.findall(r'(\d+)', num_jobs.text)
# match[1]

In [ ]:
def num_jobs_indeed(first_page_soup):
    '''
    This function returns the total number of the jobs in the searching result.
    '''
    # Find out the section contains total number of jobs  
    div = first_page_soup.find('div', id='searchCountPages')
    # Extract the number
    num_jobs = re.findall(r'(\d+)', div.text)[1]
    return num_jobs

In [ ]:
def page_num_indeed(url):
    '''
    This function returns the page number of job searching results. 
    '''
    # Create a Soup object based on the url
    soup = page_soup_indeed(url)
    # Find out the section contains total number of jobs  
    div = soup.find('div', id='searchCountPages')
    # Extract the number
    page_num = re.findall(r'(\d+)', div.text)[0]
    return page_num

In [ ]:
# Define a function to extract all job cards in a Indeed page

def job_cards_indeed(soup):
    '''
    This function accepts the Soup object of a Indeed page 
    return an iterator containing the all the job cards in this page.
    '''
    # Find the appropriate tag that contains all of the job listings in this page
    tag = soup.find('td', id="resultsCol")
    # Extract all job cards
    job_cards = tag.find_all('div', class_='jobsearch-SerpJobCard')
    return job_cards

In [ ]:
# # Test the function job_cards_indeed
# job_cards = job_cards_indeed(first_page_soup)

# # Print the data type of job_cards
# type(job_cards)

# # How many jobs listed in the 1st page? 
# len(job_cards)

In [ ]:
def job_titles_indeed(job_cards):
    '''
    This function extract the job titles from a job_cards set. 
    '''
    # Create a list to hold the job titles
    titles = []
    # For Loop throught the job cards to extract the titles
    for job in job_cards:
        title = job.find('h2', class_='title')
        title = title.text.strip()
        titles.append(title)
    return titles

In [ ]:
# Define a function to pull the company names from a set of job cards

def company_names_indeed(job_cards):
    '''
    This function extracts the company names from a set of job cards.
    '''
    # Create a list to hold the company names
    names = []
    # For loop through the job cards to pull the company names
    for job in job_cards:
        name = job.find('span', class_='company')
        name = name.text.strip()
        names.append(name)
    return names

In [ ]:
# Define a function to pull the post ages from a set of job cards

def post_ages_indeed(job_cards):
    '''
    This function pulls the post ages from a set of job cards.
    '''
    # Create a list to hold the post ages
    ages = []
    # For loop through the job cards to pull the post ages
    for job in job_cards:
        age = job.find('span', class_='date')
        age = age.text.strip()
        ages.append(age)
    return ages

In [ ]:
# Define a function to pull the location from a set of job cards

def job_locations_indeed(job_cards):
    '''
    This function pulls the job locations from a set of job cards.
    '''
    # Create a list to hold the locations
    locations = []
    # For loop through the job cards to pull the locations
    for job in job_cards:
        location = job.find('div', class_='location accessible-contrast-color-location')
        if location == None:
            location = job.find('span', class_='location accessible-contrast-color-location')
        location = location.text.strip()
        locations.append(location)
    return locations

In [ ]:
# Define a function to pull the company ratings from a set of job cards

def company_rating_indeed(job_cards):
    '''
    This function pulls the company rating from a set of job cards.
    If the rating is unavailable, it will be marked as 'missing'.
    '''
    # Create a list to hold the locations
    ratings = []
    # For loop through the job cards to pull the locations
    for job in job_cards:
        rating = job.find('span', class_='ratingsContent')
        if rating == None:
            ratings.append('missing')
            continue
        rating = rating.text.strip()
        ratings.append(rating)
    return ratings

In [ ]:
def acuqire_indeed_job_description(url):
    '''
    This function accepts the URL of a job posting and pull its description.
    '''
    # Make the HTTP request
    request = requests.get(url)
    print("Status Code: ", request.status_code)
    # Take a break
    time.sleep(5)
    # Make a soup variable holding the response content
    soup = BeautifulSoup(request.content, "html.parser")
    if soup == None:
        description = 'error'
    else:
        # Print the page's title
        print(soup.title.string)
        # Find the section that contains job description
        description = soup.find('div', id="jobDescriptionText")
        if description == None:
            description = 'error'
        else:
            description = description.text
    return description

def job_links_and_contents_indeed(job_cards):
    '''
    This function pulls the job links and descriptions from a set of job cards.
    '''
    # Create a list to hold the links and descriptions
    links = []
    descriptions = []
    # For loop through the job cards to pull the links and descriptions
    for job in job_cards:
        link = job.find('a')['href']
        link = 'https://www.indeed.com' + link
        link = link.replace(';', '&')
        description = acuqire_indeed_job_description(link)
        links.append(link)
        descriptions.append(description)
    return links, descriptions

In [ ]:
# Define a function to create a Soup object based on a job search url

def page_soup_indeed(url):
    '''
    This function returns a BeautifulSoup object to hold the content 
    of a page for a job searching results at Indeed.com
    '''
    # Make the HTTP request
    response = requests.get(url)
    # Print the status code of the request
    print("Status code of the request: ", response.status_code)
    # Sanity check to make sure the document type is HTML
    print("Document type: ", response.text[:15])
    # Take a break
    time.sleep(5)
    # Make a soup to hold the response content
    soup = BeautifulSoup(response.content, "html.parser")
    # Print out the title of the content
    print("Title of the response: ", soup.title.string)
    return soup

In [ ]:
# # Test the function: page_soup_indeed

# url = 'https://www.indeed.com/jobs?q=data+scientist&l=al&sort=date'
# soup = page_soup_indeed(url)
# type(soup)

# # Find out the page number
# int(page_num_indeed(url))

# # Pull the job cards from the soup
# type(job_cards_indeed(soup))

In [ ]:
# Define a function to pull job information from a job search URL

def acquire_page_indeed(url):
    '''
    This function accepts a job search URL and returns a pandas dataframe 
    containing job title, location, company, company rating, post age and description. 
    '''
    # Create a Soup object based on the url
    soup = page_soup_indeed(url)
    # Pull the job cards
    job_cards = job_cards_indeed(soup)
    # Pull the job titles
    titles = job_titles_indeed(job_cards)   
    # Pull the names of the companies
    companies = company_names_indeed(job_cards)
    # Pull the post ages
    ages = post_ages_indeed(job_cards)
    # Pull the job locations
    locations = job_locations_indeed(job_cards)
    # Pull the company ratings
    ratings = company_rating_indeed(job_cards)
    # Pull the hyperlinks and job description
    links, descriptions = job_links_and_contents_indeed(job_cards)    
    # Create a dataframe
    d = {'title': titles,
         'location': locations,
         'company': companies, 
         'company_rating': ratings,
         'post_age': ages, 
         'job_link': links, 
         'job_description': descriptions}
    df = pd.DataFrame(d)
    return df

In [ ]:
def jobs_indeed(job_title, location):
    '''
    This function accepts the job title and location and return 
    the job information pull from Indeed.com.
    '''
    # Generate the urls based on job title and location (state)
    url = first_page_url = first_page_url_indeed(job_title, location)
    # Set up an counter
    counter = 1
    # Create an empty dataframe to hold the job information
    df_jobs = pd.DataFrame(columns = ['title', 'location', 'company', 'company_rating', 
                                      'post_age','job_link', 'job_description'])
    # Pull the page number
    page_num = int(page_num_indeed(url))
    # Set up an checker
    keep_going = (counter == page_num)   
    # For loop through the urls to pull job information
    while keep_going and page_num <=35:
        df = acquire_page_indeed(url)
        print("--------------------------------")
        print("Page: ", page_num)
        print("--------------------------------")
        df_jobs = df_jobs.append(df, ignore_index=True)
        time.sleep(180)
        dic = {'start': page_num*10}
        relative_url = urllib.parse.urlencode(dic)
        url = first_page_url + '&' + relative_url
        counter = counter + 1
        page_num = int(page_num_indeed(url))
        keep_going = (counter == page_num)
    # Print the total number of jobs
    print(f"Total number of {job_title} positions in {location}: ", df_jobs.shape[0])
    return df_jobs

## Data Preparation

In [188]:
# Define a function to remove the duplicates

def remove_duplicates(df):
    '''
    This function removes the duplicates in the dataframe
    '''
    # Define the columns for identifying duplicates
    columns = ['title', 'location', 'company', 'job_link', 'job_description']
    # Drop the duplicates except for the last occurrence
    df.drop_duplicates(subset=columns, inplace=True, keep='last')
    return df

In [189]:
# Define a function to compute the date of the job posts

def compute_post_date(df):
    '''
    This function computes the date of the job post based on post age
    and set the date as the index of the dataframe.
    '''
    # Create an empty list to hold the post date
    post_date = []
    # For loop the column post_age and convert the values to date
    for age in df.post_age:
        if age == 'Just posted':
            date = datetime.date.today()
            post_date.append(date)
        elif age == 'Today':
            date = datetime.date.today()
            post_date.append(date)
        else:
            # Extract the number
            num = re.findall(r'(\d+)', age)[0]
            # Cast the string number to integer
            num = int(num)
            # Convert the integer to timedelta object
            num = datetime.timedelta(days=num)
            # Compute post date        
            date = datetime.date.today()
            date = date - num
            post_date.append(date)
    # Add post date as new column
    df['date'] = post_date
    # Set the column post_date as the index and sort the values
    df = df.set_index('date').sort_index(ascending=False)
    return df

In [190]:
# Define a function to transform old job posts files

def transform_old_file(df, date_string):
    '''
    This function accepts old daily job posts and convert the post age to post date. 
    '''
    # Create an empty list to hold the post date
    post_date = []
    # For loop the column post_age and convert the values to date
    for age in df.post_age:
        if age == 'Just posted':
            date = datetime.date.fromisoformat(date_string)
            post_date.append(date)
        elif age == 'Today':
            date = datetime.date.fromisoformat(date_string)
            post_date.append(date)
        else:
            # Extract the number
            num = re.findall(r'(\d+)', age)[0]
            # Cast the string number to integer
            num = int(num)
            # Convert the integer to timedelta object
            num = datetime.timedelta(days=num)
            # Compute post date        
            date = datetime.date.fromisoformat(date_string)
            date = date - num
            post_date.append(date)
    # Add post date as new column
    df['date'] = post_date
    # Set the column post_date as the index and sort the values
    df = df.set_index('date').sort_index(ascending=False)
    return df

### Web Deveopment

In [191]:
# Load web developer job posts in TX today

# Import the file path
database = env_Shi.database

# Read the daily data scientist jobs in TX
df_wd_new = pd.read_csv(f"{database}web_developer_tx_indeed_020921.csv", index_col=0)

# Print the dimentionality
print(df_wd_new.shape)

# Print the first two rows
df_wd_new.head(2)

(540, 7)


,title,location,company,company_rating,post_age,job_link,job_description
0,"Senior Software Engineer, Front End\nnew","Austin, TX 78701 (Downtown area)",eBay Inc.,3.9,Just posted,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,"At eBay, you will be part of a purpose driven ..."
1,AWS Developer\nnew,"Houston, TX",TCS,3.8,Just posted,https://www.indeed.com/rc/clk?jk=210bc31caa4af...,"Mandatory Technical Skills* C# , .Net Core, We..."


In [192]:
def daily_update_wd(df):
    '''
    This function updates job posts of web developer in TX by adding the daily acquring
    of web developer job posts in TX. 
    '''
    # Read the job posts of web developer in TX
    database = env_Shi.database
    df_wd_tx = pd.read_csv(f"{database}df_wd_tx_backup.csv")
    num_jobs = df_wd_tx.shape[0]
    # Convert the date column to datetime type
    df_wd_tx.date = pd.to_datetime(df_wd_tx.date)
    # Set the date column as the index and sort the index
    df_wd_tx = df_wd_tx.set_index('date').sort_index(ascending=False)
    # Add the daily update
    df = compute_post_date(df)
    df_wd_tx = pd.concat([df_wd_tx, df]).sort_index(ascending=False)
    # Remove the duplicates
    df_wd_tx = remove_duplicates(df_wd_tx)
    # Save as csv file
    df_wd_tx.to_csv(f"{database}df_wd_tx_backup.csv")
    num_new_jobs = df_wd_tx.shape[0] - num_jobs
    print("New Jobs Posted Today: ", num_new_jobs)
    return df_wd_tx

In [193]:
# Test function: daily_update_wd

df_test = daily_update_wd(df_wd_new)
df_test.head(2)

New Jobs Posted Today:  189


,title,location,company,company_rating,post_age,job_link,job_description
date,,,,,,,
2021-02-09,Senior Software Developer\nnew,"Austin, TX",Emerson,3.9,Today,https://www.indeed.com/rc/clk?jk=805ea6fc9112a...,Senior Software Developer\n\nRequisition ID: (...
2021-02-09,Web site Developer/Manager\nnew,"San Antonio, TX 78216",Save Merchant Services,missing,Today,https://www.indeed.com/company/Save-Merchant-S...,We are seeking a highly qualified web site des...


In [194]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2842 entries, 2021-02-09 to 2021-01-04 00:00:00
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   title            2842 non-null   object
 1   location         2842 non-null   object
 2   company          2842 non-null   object
 3   company_rating   2842 non-null   object
 4   post_age         2842 non-null   object
 5   job_link         2842 non-null   object
 6   job_description  2842 non-null   object
dtypes: object(7)
memory usage: 177.6+ KB


In [195]:
# Define a function to prepare the job posts of web developer

def prepare_job_posts_indeed_wd():
    '''
   The function cleans the csv file of web developer job posts and save as json. 
    '''
    # Read the job posts of web developer in TX
    database = env_Shi.database
    df = pd.read_csv(f"{database}df_wd_tx_backup.csv")
    # Create columns of city, state, and zipcode
    location = df.location.str.split(', ', expand=True)
    location.columns = ['city', 'zipcode']
    location.city = location.city.apply(lambda i: 0 if i == 'United States' else i)
    location.city = location.city.apply(lambda i: 0 if i == 'Texas' else i)
    location.zipcode = location.zipcode.apply(lambda i: 0 if re.findall(r"(\d+)", str(i)) == [] 
                                          else re.findall(r"(\d+)", str(i))[0])
    df['city'] = location.city
    df['state'] = 'TX'
    df['zipcode'] = location.zipcode
    # Replace the missing values in the company rating with 0
    df.company_rating = df.company_rating.apply(lambda i: 0 if i == 'missing' else i)
    # Drop the column post_age and location
    df = df.drop(columns=['post_age', 'location'])
    # Clean the text in the job description
    df = MVP_Bojado.prep_job_description_data(df, 'job_description')
    # Save a JSON version of the prepared data
    df.to_json(f"{database}df_wd_tx_prepared_backup.json", orient='records')
    return df

In [196]:
%%time

# Test the function: prepare_job_posts_indeed_wd
df_test = prepare_job_posts_indeed_wd()
df_test.head(2)

CPU times: user 32.6 s, sys: 346 ms, total: 33 s
Wall time: 33.1 s


,date,title,company,company_rating,job_link,job_description,city,state,zipcode,clean,tokenized,stemmed,lemmatized
0,2021-02-09,Senior Software Developer\nnew,Emerson,3.9,https://www.indeed.com/rc/clk?jk=805ea6fc9112a...,Senior Software Developer\n\nRequisition ID: (...,Austin,TX,0,senior software developer requisition id 20008...,senior software developer\n\nrequisition id 20...,senior softwar develop requisit id 20008701 pr...,senior software developer requisition id 20008...
1,2021-02-09,Web site Developer/Manager\nnew,Save Merchant Services,0,https://www.indeed.com/company/Save-Merchant-S...,We are seeking a highly qualified web site des...,San Antonio,TX,78216,seeking highly qualified web site designer hig...,we are seeking a highly qualified web site des...,we are seek a highli qualifi web site design w...,we are seeking a highly qualified web site des...


In [197]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2842 entries, 0 to 2841
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   date             2842 non-null   object
 1   title            2842 non-null   object
 2   company          2842 non-null   object
 3   company_rating   2842 non-null   object
 4   job_link         2842 non-null   object
 5   job_description  2842 non-null   object
 6   city             2842 non-null   object
 7   state            2842 non-null   object
 8   zipcode          2842 non-null   object
 9   clean            2842 non-null   object
 10  tokenized        2842 non-null   object
 11  stemmed          2842 non-null   object
 12  lemmatized       2842 non-null   object
dtypes: object(13)
memory usage: 288.8+ KB


In [198]:
# Define the columns for identifying duplicates
columns = ['date', 'title', 'company', 'job_link', 'job_description', 'city', 'state', 'zipcode']
   
# Check for duplicates
duplicates = df_test.duplicated(subset=columns,keep='last')
duplicates.sum()

0

In [ ]:
# Read the json file

result = open(f"{database}df_wd_tx_prepared_backup.json")
parsed = json.load(result)
parsed[0]

#### Data Scientist

In [ ]:
# # Load old data scientist job posts in TX

# # Import the file path
# database = env_Shi.database

# # Read the daily data scientist jobs in TX
# df_ds_old = pd.read_csv(f"{database}data_scientist_tx_indeed_020821.csv", index_col=0)

# # Print the first 2 rows
# df_ds_old.head(2)

# # Transform old file
# df_test = transform_old_file(df_ds_old, '2021-02-08')
# df_test.head(2)

In [ ]:
# Load data scientist job posts in TX on 2021-02-08

# Import the file path
database = env_Shi.database

# Read the daily data scientist jobs in TX
df_ds_new = pd.read_csv(f"{database}data_scientist_tx_indeed_020821.csv", index_col=0)

# Inspect the first 2 rows of the new posts
df_ds_new.head(2)

In [ ]:
# Test the function: compute_post_date

df_test = compute_post_date(df_ds_new)
df_test.head(2) # Works

In [ ]:
def daily_update_ds(df):
    '''
    This function updates job posts of data scientist in TX by adding the daily acquring
    of data scientist job posts in TX. 
    '''
    # Read the job posts of data scientist in TX
    database = env_Shi.database
    df_ds_tx = pd.read_csv(f"{database}df_ds_tx_backup.csv")
    num_jobs = df_ds_tx.shape[0]
    # Convert the date column to datetime type
    df_ds_tx.date = pd.to_datetime(df_ds_tx.date)
    # Set the date column as the index and sort the index
    df_ds_tx = df_ds_tx.set_index('date').sort_index(ascending=False)
    # Add the daily update
    df = compute_post_date(df)
    df_ds_tx = pd.concat([df_ds_tx, df]).sort_index(ascending=False)
    # Remove the duplicates
    df_ds_tx = remove_duplicates(df_ds_tx)
    # Save as csv file
    df_ds_tx.to_csv(f"{database}df_ds_tx_backup.csv")
    # Print the new jobs posted today
    num_new_jobs = df_ds_tx.shape[0] - num_jobs
    print("New Jobs Posted Today: ", num_new_jobs)
    return df_ds_tx

In [ ]:
# Test the function: daily_update_ds

df_test = daily_update_ds(df_ds_new)
df_test.head() # Works

In [ ]:
# Print the information of the dateframe
df_test.info()

In [ ]:
# Define a function to prepare the job post for exploration

def prepare_job_posts_indeed_ds():
    '''
    The function cleans the csv file of data scientist job posts and save as json. 
    '''
    # Read the job posts of data scientist in TX
    database = env_Shi.database
    df = pd.read_csv(f"{database}df_ds_tx_backup.csv")
    # Create columns of city, state, and zipcode
    location = df.location.str.split(', ', expand=True)
    location.columns = ['city', 'zipcode']
    location.city = location.city.apply(lambda i: 0 if i == 'United States' else i)
    location.city = location.city.apply(lambda i: 0 if i == 'Texas' else i)
    location.zipcode = location.zipcode.apply(lambda i: 0 if re.findall(r"(\d+)", str(i)) == [] 
                                          else re.findall(r"(\d+)", str(i))[0])
    df['city'] = location.city
    df['state'] = 'TX'
    df['zipcode'] = location.zipcode
    # Replace the missing values in the company rating with 0
    df.company_rating = df.company_rating.apply(lambda i: 0 if i == 'missing' else i)
    # Drop the column post_age and location
    df = df.drop(columns=['post_age', 'location'])
    # Clean the text in the job description
    df = MVP_Bojado.prep_job_description_data(df, 'job_description')
    # Save a JSON version of the prepared data
    df.to_json(f"{database}df_ds_tx_prepared_backup.json", orient='records')
    return df

In [ ]:
%%time
# Test the function: prepare_job_posts_indeed

df_test = prepare_job_posts_indeed_ds()
df_test.head(2)

In [ ]:
df_test.info()

In [ ]:
# Define the columns for identifying duplicates
columns = ['date', 'title', 'company', 'job_link', 'job_description', 'city', 'state', 'zipcode']
   
# Check for duplicates
duplicates = df_test.duplicated(subset=columns,keep='last')
duplicates.sum()

In [ ]:
# Read the json file
database = env_Shi.database
result = open(f"{database}df_ds_tx_prepared_backup.json")

# Print the type of the file
print(type(result))

# 
parsed = json.load(result)
parsed[0]

# Data Exploration

In [ ]:
# # import 
# s3 = boto3.resource('s3')

# # Print the data type of s3
# print(type(s3))

# # Print the bucket names
# for bucket in s3.buckets.all():
#     print(bucket.name)

In [200]:
# Read the json file
database = env_Shi.database
df_ds_tx = pd.read_json(f"{database}df_ds_tx_prepared_backup.json")

# Print the number of job posts
print("Number of Job Post: ", df_ds_tx.shape[0])

# Conver the string date to datetime object
df_ds_tx.date = pd.to_datetime(df_ds_tx.date)

# Set the date as the index and sort the dataframe in descending order
df_ds_tx = df_ds_tx.set_index('date').sort_index(ascending=False)
df_ds_tx.head()

Number of Job Post:  1501


,title,company,company_rating,job_link,job_description,city,state,zipcode,clean,tokenized,stemmed,lemmatized
date,,,,,,,,,,,,
2021-02-08,Senior Data Analyst\nnew,Citizens,3.4,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Description\nThe Senior Data Analyst is adept ...,Hurst,TX,76054,description senior data analyst adept working ...,description\nthe senior data analyst is adept ...,descript the senior data analyst is adept at w...,description the senior data analyst is adept a...
2021-02-08,Data Scientist Senior - Computer Vision/Deep L...,USAA,3.9,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Purpose of Job We are currently seeking a tale...,LaCoste,TX,78039,purpose job currently seeking talented data sc...,purpose of job we are currently seeking a tale...,purpos of job we are current seek a talent dat...,purpose of job we are currently seeking a tale...
2021-02-08,Senior Catastrophe Modeling Analyst\nnew,USAA,3.9,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Purpose of Job We are currently seeking a Seni...,San Antonio,TX,78206,purpose job currently seeking senior catastrop...,purpose of job we are currently seeking a seni...,purpos of job we are current seek a senior cat...,purpose of job we are currently seeking a seni...
2021-02-08,Data Scientist - 100% Remote Available\nnew,USAA,3.9,https://www.indeed.com/cmp/Usaa,error,Shavano Park,TX,78231,error,error,error,error
2021-02-08,Risk - Corporate Risk - Wholesale Credit Solut...,"JPMorgan Chase Bank, N.A.",3.9,https://www.indeed.com/rc/clk?jk=2f4b1ed986adf...,Organization\nJPMorgan Chase & Co. (NYSE: JPM)...,Plano,TX,0,organization jpmorgan chase co nyse jpm leadin...,organization\njpmorgan chase co nyse jpm is a ...,organ jpmorgan chase co nyse jpm is a lead glo...,organization jpmorgan chase co nyse jpm is a l...


In [201]:
# Print the information of the df_ds_tx
df_ds_tx.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1501 entries, 2021-02-08 to 2020-12-22
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   title            1501 non-null   object 
 1   company          1501 non-null   object 
 2   company_rating   1501 non-null   float64
 3   job_link         1501 non-null   object 
 4   job_description  1501 non-null   object 
 5   city             1501 non-null   object 
 6   state            1501 non-null   object 
 7   zipcode          1501 non-null   int64  
 8   clean            1501 non-null   object 
 9   tokenized        1501 non-null   object 
 10  stemmed          1501 non-null   object 
 11  lemmatized       1501 non-null   object 
dtypes: float64(1), int64(1), object(10)
memory usage: 152.4+ KB


In [202]:
# Print the top 5 companies by the number of posts
df_ds_tx.company.value_counts().head()

Cognizant Technology Solutions    55
Dell Technologies                 40
Deloitte                          32
Facebook                          30
USAA                              28
Name: company, dtype: int64

In [203]:
# Print the top 5 cities by the number of posts
df_ds_tx.city.value_counts().head()

Austin         420
Dallas         239
Houston        190
Plano          116
San Antonio    116
Name: city, dtype: int64

In [204]:
# Sanity check: the dataframe has datetime index
df_ds_tx.resample("W").title.count()

date
2020-12-27    392
2021-01-03    136
2021-01-10    179
2021-01-17    124
2021-01-24    291
2021-01-31    251
2021-02-07    123
2021-02-14      5
Freq: W-SUN, Name: title, dtype: int64

### Extract Job Requirements by Regular Expression

In [ ]:
# Take a random job link

job_url = df_ds.job_link.sample(1, random_state=1)[0]
job_url

In [ ]:
# Make the rquest

response = requests.get(job_url)
response.status_code

In [ ]:
# Make a soup to hold the response content
soup = BeautifulSoup(response.content, 'html.parser')
soup.title.string

In [ ]:
soup.style

In [ ]:
print(soup.prettify())

In [ ]:
# Create 'words' variable
words = [re.sub(r'([^a-z0-9\s]|\s.\s)', '', doc).split() for doc in df_ds_tx.clean]

# Add 'words' column to dataframe
# Column will contain lists of separated words in each repo
df_ds_tx = pd.concat([df_ds_tx, pd.DataFrame({'words': words})], axis=1)

df_ds_tx.head(2)

## Frequency Analysis of Mono-, Bi-, and Tri-grams

### Create a list of all the words appear in the job descriptions

In [205]:
# Define the function to create the words that appear in the job descriptions

def words_variables_v1(df):
    '''
    This function accepts the dataframe with cleaned job description 
    and return a dictionary in which the values are the words that 
    appear in the job description. 
    '''
    # Create the words that appear all the job descritipons
    all_words = ' '.join(df.clean)
    # Create a dictionary to hold the variable all_words
    d_words = {'frequency': all_words}
    return d_words

In [206]:
# Test the helper function: words_variables_v1
dic = words_variables_v1(df_ds_tx)

# Print out the keys
print(dic.keys())

# Print the first 100 characters of the value
dic['frequency'][:100]

dict_keys(['frequency'])


'description senior data analyst adept working business system partner turn project requirement techn'

In [ ]:
# Upgrade the function `words_variables_v1`

def words_variables_v2(df, companies):
    '''
    This function accepts the dataframe containing cleaned job description and 
    a list of company names and return a dictionary in which the values are the words 
    that appear in the job description. 
    '''
    # Create the words that appear all the job descritipons
    all_words = ' '.join(df.clean)
    # Create a dictionary to hold the variable all_words
    d_words = {'all': all_words}
    # For loop the companies and create the words that appear in their job descriptions
    for company in companies:
        mask = (df.company == company)
        s_company = df[mask].clean
        words = ' '.join(s_company)
        d_words[company] = words
    return d_words

In [ ]:
# Test the helper function: words_variables_v2

companies = ['Apple']
dic_v2 = words_variables_v2(df_ds_tx, companies)

# Print out the keys
print(dic_v2.keys())

# Print the first 100 characters of the value of `Apple`
dic_v2['Apple'][:400]

### Monogram Analysis

In [ ]:
# Define a function to compute the word frequency in the job description

def word_frequency_v1(d_words):
    '''
    This function accept the dictionary created by function words_variables_v1
    and return the word frequency in the job description. 
    '''
    # Create a dataframe to hold the word frequency
    word_counts = pd.DataFrame()
    # Compute the words frequency
    freq = pd.Series(d_words['frequency'].split()).value_counts()
    word_counts = pd.concat([word_counts, freq], axis=1, sort=True)
    word_counts.columns = d_words.keys()
    word_counts.sort_values(by='frequency', ascending=False, inplace=True)
    return word_counts

In [ ]:
# Upgrade `word_frequency_v1`

def word_frequency_v2(d_words):
    '''
    This function accept the dictionary created by function words_variables_v2
    and return the word frequency in the job description. 
    '''
    # Read the company names from the dictionary
    companies = d_words.keys()
    # Create a dataframe to hold the word frequency
    word_counts = pd.DataFrame()
    # For loop through the companies and generate the word frequency in their job descriptions
    for company in companies:
        freq = pd.Series(d_words[company].split()).value_counts()
        word_counts = pd.concat([word_counts, freq], axis=1, sort=True)
    word_counts.columns = companies
    word_counts = word_counts.fillna(0).apply(lambda s: s.astype(int))
    word_counts.sort_values(by='all', ascending=False, inplace=True)
    return word_counts

In [ ]:
# Test the function word_frequency_v1

monogram = word_frequency_v1(dic)
monogram.head(5)

In [ ]:
monogram.info()

### Bigram Analysis

In [ ]:
# Test the function word_frequency_v2

df_word_frequency_v2 = word_frequency_v2(dic_v2)
df_word_frequency_v2.head(5)

In [ ]:
# Added 'Bigram' column to dataframe
df_ds_tx['bigrams'] = [list(nltk.ngrams(wordlist, 2)) for wordlist in df_ds_tx.words]
df_ds_tx.head(2)

In [ ]:
def bigrams_frequency_v1(d_words):
    '''
    This function accept the dictionary created by function words_variables_v1
    and return the word frequency in the job description. 
    '''
    # Create a dataframe to hold the word frequency
    word_counts = pd.DataFrame()
    # Compute the words frequency
    freq = pd.Series(list(nltk.ngrams(d_words['frequency'].split(), 2))).value_counts()
    # Add the `freq` seires to `word_counts` dataframe
    word_counts = pd.concat([word_counts, freq], axis=1, sort=True)
    # Rename the coumns
    word_counts.columns = d_words.keys()
    # Sort the dataframe by the values in column `frequency`
    word_counts.sort_values(by='frequency', ascending=False, inplace=True)
    return word_counts

In [ ]:
bigrams = bigrams_frequency_v1(dic)
bigrams.head()

In [ ]:
# Define a function to compute the bigrams frequency in the job description

def bigrams_frequency_v2(d_words):
    '''
    This function accept the dictionary created by function words_variables_v2
    and return the bigrams frequency in the job description. 
    '''
    # Read the company names from the dictionary
    companies = d_words.keys()
    # Create a dataframe to hold the word frequency
    bigrams_counts = pd.DataFrame()
    # For loop through the companies and generate the word frequency in their job descriptions
    for company in companies:
        freq = pd.Series(list(nltk.ngrams(d_words[company].split(), 2))).value_counts()
        bigrams_counts = pd.concat([bigrams_counts, freq], axis=1, sort=True)
    bigrams_counts.columns = companies
    bigrams_counts = bigrams_counts.fillna(0).apply(lambda s: s.astype(int))
    bigrams_counts.sort_values(by='all', ascending=False, inplace=True)
    return bigrams_counts

In [ ]:
# Compute bigrams_frequency

bigrams_v2 = bigrams_frequency_v2(dic_v2)
bigrams_v2.head()

### Trigram Analysis

In [ ]:
def trigrams_frequency_v1(d_words):
    '''
    This function accept the dictionary created by function words_variables_v1
    and return the word frequency in the job description. 
    '''
    # Create a dataframe to hold the word frequency
    word_counts = pd.DataFrame()
    # Compute the words frequency
    freq = pd.Series(list(nltk.ngrams(d_words['frequency'].split(), 3))).value_counts()
    # Add the `freq` seires to `word_counts` dataframe
    word_counts = pd.concat([word_counts, freq], axis=1, sort=True)
    # Rename the coumns
    word_counts.columns = d_words.keys()
    # Sort the dataframe by the values in column `frequency`
    word_counts.sort_values(by='frequency', ascending=False, inplace=True)
    return word_counts

In [ ]:
# Test function: trigrams_frequency_v1

trigrams = trigrams_frequency_v1(dic)
trigrams.head()

In [ ]:
# Define a function to compute the trigrams frequency in the job description

def trigrams_frequency_v2(d_words):
    '''
    This function accept the dictionary created by function words_variables_v2
    and return the trigrams frequency in the job description. 
    '''
    # Read the company names from the dictionary
    companies = d_words.keys()
    # Create a dataframe to hold the word frequency
    trigrams_counts = pd.DataFrame()
    # For loop through the companies and generate the word frequency in their job descriptions
    for company in companies:
        freq = pd.Series(list(nltk.ngrams(d_words[company].split(), 3))).value_counts()
        trigrams_counts = pd.concat([trigrams_counts, freq], axis=1, sort=True)
    trigrams_counts.columns = companies
    trigrams_counts = trigrams_counts.fillna(0).apply(lambda s: s.astype(int))
    trigrams_counts.sort_values(by='all', ascending=False, inplace=True)
    return trigrams_counts

In [ ]:
# Test function: trigrams_frequency_v2

trigrams_v2 = trigrams_frequency_v2(dic_v2)
trigrams_v2.head()

### Combine Mono-, Bi- and Trigrams

#### Method 1: Simple concatenation

In [ ]:
# What is total number of grams? 

monogram.shape[0]+bigrams.shape[0]+trigrams.shape[0]

In [ ]:
# Concat all three grams

everygram = pd.concat([monogram, bigrams, trigrams])
print(everygram.shape)
everygram.sample(20)

#### Method 2:  Use nltk.util.everygrams

In [207]:
# Define a function to compute the frequence of the mono-, bi-, and tri-grams of the job description

def everygram_frequency_v1(d_words, max_len=3):
    '''
    This function accetps the dictionary produced by the function `words_variables_v1` and 
    return mono-, bi-, and tri-grams along with their frequency. 
    '''
    # Generate mono-, bi-, and tri-grams
    grams = nltk.everygrams(d_words['frequency'].split(), max_len=max_len) # dtype of grams: <class 'genertor'>
    # Convert to a list of tuples
    grams = list(grams)
    # Create an empty list to hold mono-, bi-, and tri-grams
    everygram = []
    # For loop the list of tuples and convert the grams to strings
    for gram in grams:
        str_gram = gram[0]
        for i in gram[1:]:
            str_gram = str_gram + ' ' + i
        everygram.append(str_gram)
    # Compute the frequency of the everygrams
    everygram = pd.Series(everygram).value_counts()
    return everygram

In [208]:
# Test the function above

everygram = everygram_frequency_v1(dic)
everygram.head()

data          15473
experience     8540
business       5449
team           4783
work           4219
dtype: int64

In [209]:
# Define a function to pick up the top k skills of a data scientict from a skillset library

def top_skills_ds_v1(k, library):
    '''
    This function accepts a positive integer k and a skillset library and 
    returns a dataframe containing the top k skills needed for data scientist positions.
    '''
    # Import the file path
    database = env_Shi.database
    # Load the prepared dataframe with job search results
    df = pd.read_json(f"{database}df_ds_tx_prepared_backup.json")
    # Create a string of all words that appear in the job description
    dic = words_variables_v1(df)
    # Compute the words frequency
    everygram_frequency = everygram_frequency_v1(dic)
    # Create a empty dataframe to hold the rank of the skills
    df_skills = pd.DataFrame()
    # For loop through the library to find out the frequency of the skills mentioned in the job description
    for skill in library:
        mask = ( everygram_frequency.index == skill)
        df =  everygram_frequency[mask]
        df_skills = pd.concat([df_skills, df])
    df_skills.columns = dic.keys()
    df_skills.sort_values(by='frequency', ascending=False, inplace=True)
    return df_skills.head(k)

In [210]:
# Define a sample library of skills for data scientist
library = ['python', 'sql', 'critical thinking', 'data scientist', 'verbal written communication']

# Test function: top_skills_ds_v1
df_test = top_skills_ds_v1(5, library)
df_test

,frequency
python,1249.0
data scientist,1043.0
sql,948.0
verbal written communication,123.0
critical thinking,69.0


### Skills Match Job Search

In [ ]:
# Create the masks for different skills

mask_python = df_ds_tx.clean.str.contains('python')
mask_sql = df_ds_tx.clean.str.contains('sql')
mask_ml = df_ds_tx.clean.str.contains('machine learning')
mask_tableau = df_ds_tx.clean.str.contains('tableau')
mask_aws = df_ds_tx.clean.str.contains('aws')

mask = mask_python & mask_sql & mask_tableau

In [ ]:
# How many companies need all three skills: python, sql and tableau
mask.sum()

In [ ]:
df_ds_tx[mask].head(1)

In [ ]:
df_ds_tx.clean[0][:100]

### Compute Top 5 Skills in a Predifined Library

In [ ]:
# Create a library for all skills

library = ['python', 'r', 'sql', 'tableau', 'scikitlearn', 'tensorflow', 'pytorch', 
           'aws', 'hadoop', 'hive', 'impala', 'matlab', 'model', 'algorithm', 
           'storytelling', 'statistic', 'etl', 'exploration', 'extraction', 
           'sharepoint', 'dashboard']

library_tech = ['programming', 'big data', 'wrangling', 'version control', 'visualiztion', ]
library_soft = ['communication', 'business acumen', 'storytelling']
library_tools = ['python', 'git', 'sql', 'pandas']

In [ ]:
# data visualization
# big data
# software engineering
# model
# models
# algorithms
# storytelling
# statistic
# statistical
# machine learning
# deep learning
# etl
# extraction
# crud
# exploration

In [ ]:
def top_skills_ds_v1(k):
    '''
    This function accepts a positive integer k and 
    returns a dataframe containing the top k skills needed
    for data scientist positions.
    '''
    # Import the file path
    database = env_Shi.database
    # Load the prepared dataframe with job search results
    df = pd.read_csv(f"{database}df_tx_ds.csv", index_col=0)
    # Create a string of all words that appear in the job description
    dic = words_variables_v1(df)
    # Compute the words frequency
    df_word_frequency = word_frequency_v1(dic)
    # Define a library that has a complete sillset for data scientist
    library = ['python', 'r', 'sql', 'tableau', 'scikitlearn', 'tensorflow', 'pytorch', 'aws', 'hadoop', 'hive', 
        'impala', 'matlab', 'model', 'algorithm', 'storytelling', 'statistic', 'etl', 'exploration', 'extraction', 
        'sharepoint', 'dashboard']
    # Create a empty dataframe to hold the rank of the skills
    df_skills = pd.DataFrame()
    # For loop through the library to find out the frequency of the skills mentioned in the job description
    for skill in library:
        mask = (df_word_frequency.index == skill)
        df = df_word_frequency[mask]
        df_skills = pd.concat([df_skills, df])
    df_skills.sort_values(by='frequency', ascending=False, inplace=True)
    return df_skills.head(k)

In [ ]:
# Test function top_skills_ds

top_skills = top_skills_ds_v1(7)
top_skills

In [ ]:
mask = (df_word_frequency.index == 'python')
df_word_frequency[mask]

In [ ]:
mask = (df_word_frequency.index == 'r')
df_word_frequency[mask].sort_values(by='all', ascending=False).head(10)

In [ ]:
mask = (df_word_frequency.index == 'aws')
df_word_frequency[mask].sort_values(by='all', ascending=False).head(10)

In [ ]:
mask = (df_word_frequency.index == 'sql')
df_word_frequency[mask].sort_values(by='all', ascending=False).head(10)

In [ ]:
### Test git push